In [3]:
import cv2
import os

# path = '/content/video.mp4'
# Read the video from specified path
def video_to_frame(path):
  cam = cv2.VideoCapture(path)

  try:
	
	# creating a folder named data
	  if not os.path.exists('data'):
		  os.makedirs('data')

  # if not created then raise error
  except OSError:
	  print ('Error: Creating directory of data')

  # frame
  currentframe = 0

  while(True):
	
	  # reading from frame
	  ret,frame = cam.read()

	  if ret:
		  # if video is still left continue creating images
		  name = './data/frame' + str(currentframe) + '.jpg'
		  print ('Creating...' + name)

		  # writing the extracted images
		  cv2.imwrite(name, frame)

		  # increasing counter so that it will
		  # show how many frames are created
		  currentframe += 1
	  else:
		  break

  # Release all space and windows once done
#   cam.release()
#   cv2.destroyAllWindows()

The above code will break the video into image

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from PIL import Image

def resize_image(image_path):
    """
    Resizes an image to 64x64 pixels and saves it under the same filename.

    Args:
        image_path (str): Path to the image file.

    Returns:
        None
    """
    with Image.open(image_path) as img:
        # Resize the image to 64x64 pixels
        img_resized = img.resize((64, 64))

        # Save the resized image under the same filename
        img_resized.save(image_path)

In [8]:
pwd

'/kaggle/working'

In [9]:
train_dir = "/kaggle/input/ucf-crime-dataset/Train"
test_dir = "/kaggle/input/ucf-crime-dataset/Test"

In [10]:
SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 46      #Was 46
EPOCHS = 1
LR =  0.00003
NUM_CLASSES = 14

In [11]:
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']

In [12]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input

In [13]:
train_datagen = ImageDataGenerator(
                                   #horizontal_flip=True,
                                   #width_shift_range=0.1,
                                   #height_shift_range=0.05,
                                   rescale = 1./255,
                                   validation_split=0.2, # set validation split, Added
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function=preprocess_fun
                                 )

In [14]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                     subset='training',    #Added
                                                    seed = SEED
                                                   )

Found 1013081 images belonging to 14 classes.


In [15]:
validation_generator = train_datagen.flow_from_directory(train_dir, # same directory as training data
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    subset='validation', # set as validation data,  #Added
                                                    seed = SEED
                                                   )

Found 253264 images belonging to 14 classes.


In [16]:
test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = SEED
                                                  )

Found 111308 images belonging to 14 classes.


In [17]:
def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.DenseNet121(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3),
                                               include_top=False,
                                               weights="imagenet")(inputs)
    
    return feature_extractor

In [18]:
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4) (x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)
    
    return x


In [19]:
def final_model(inputs):
    densenet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(densenet_feature_extractor)
    
    return classification_output


In [20]:
def define_compile_model():
    
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT ,IMG_WIDTH,3))
    classification_output = final_model(inputs) 
    model = tf.keras.Model(inputs=inputs, outputs = classification_output)
     
    model.compile(optimizer=tf.keras.optimizers.SGD(LR), 
                loss='categorical_crossentropy',
                metrics = [tf.keras.metrics.AUC()])
  
    return model

In [21]:
model = define_compile_model()

29084464/29084464 [==============================] - 2s 0us/step


In [22]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 densenet121 (Functional)    (None, 2, 2, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1024)              263168    
                                                             

In [ ]:
history = model.fit(x = train_generator,validation_data=validation_generator,epochs = EPOCHS)

 7178/22024 [========>.....................] - ETA: 6:24:31 - loss: 1.3609 - auc: 0.8953

In [ ]:
preds = model.predict(test_generator)
y_test = test_generator.classes
fig, c_ax = plt.subplots(1,1, figsize = (15,8)

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    for (idx, c_label) in enumerate(CLASS_LABELS):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr,lw=2, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'black',linestyle='dashed', lw=4, label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
print('ROC AUC score:', multiclass_roc_auc_score(y_test , preds  , average = "micro"))
plt.xlabel('FALSE POSITIVE RATE', fontsize=18)
plt.ylabel('TRUE POSITIVE RATE', fontsize=16)
plt.legend(fontsize = 11.5)
plt.show()

function for image risizing

In [ ]:
from PIL import Image

def resize_image(image_path):
    """
    Resizes an image to 64x64 pixels and saves it under the same filename.

    Args:
        image_path (str): Path to the image file.

    Returns:
        None
    """
    with Image.open(image_path) as img:
        # Resize the image to 64x64 pixels
        img_resized = img.resize((64, 64))

        # Save the resized image under the same filename
        img_resized.save(image_path)